In [14]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv()

True

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


"""
The add_messages method executes:

During state updates: When a node returns a dictionary with a "messages" key, the add_messages method is automatically called to combine the new messages with existing messages in the state.

Between node transitions: As the graph flows from one node to another, when state is passed between nodes, the annotations are applied to merge returned values with existing state values.

Specifically at merge points: When multiple branches of execution need to be merged (like after parallel tool execution), the annotations determine how those values should be combined.
"""
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

checkpointer = MemorySaver()

graph_builder = StateGraph(State)


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."


@tool
def broken_api(location: str):
    """Call to get the current weather."""
    return f"Currently no weather data available for {location}. Please try again later"


tools = [get_weather, broken_api]
llm = ChatOllama(
        model="mistral",
        temperature=0,
    )
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [16]:
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

""" Sets up conditional edges using tools_condition (a prebuilt function that routes to tools if needed) """
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# Adds an edge from tools back to chatbot
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

""" 
Importantly, adds interrupt_before=["tools"] which is the key for human intervention 
When you use interrupt_before=["tools"], you're telling LangGraph to pause execution right before the "tools" node would be executed.
"""
graph = graph_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["tools"],
)

In [17]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Hello, I am John")
# this won't even need to call tools so graph won't be paused.

graph.invoke({"messages": input_message}, config=config)

In [ ]:
""" 
These lines below:  

Create two separate conversations with different thread IDs
Show how the same question gets different answers because of conversation memory
Use the MemorySaver to maintain state between interactions
"""


config = {"configurable": {"thread_id": "2"}}
input_message = HumanMessage(content="Sorry, did I already introduce myself?")
# this won't even need to call tools so graph won't be paused.

graph.invoke({"messages": input_message}, config=config)

In [ ]:

config = {"configurable": {"thread_id": "1"}}
input_message = HumanMessage(content="Sorry, did I already introduce myself?")
# this won't even need to call tools so graph won't be paused.

graph.invoke({"messages": input_message}, config=config)

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "3"}}
input_message = HumanMessage(content="How is the weather in Munich?")

graph.invoke({"messages": input_message}, config=config)

In [ ]:
""" The system is essentially saying: "For conversation thread 3, I've paused execution and the next step will be to run the tools node once you tell me to continue." """

snapshot = graph.get_state(config)
snapshot.next

In [ ]:
graph.invoke(None, config=config) #resume

""" 
This demonstrates:

Starting a weather query conversation
The graph pausing before executing tools because of interrupt_before=["tools"]
Resuming execution with no new input
"""

### Timetravel

In [ ]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="How is the weather in Munich?")

graph.invoke({"messages": input_message}, config=config)

"""
This code:

Creates a new conversation with thread ID "4"
Sends a message asking about the weather in Munich
Runs the graph with this input

Since the graph was compiled with interrupt_before=["tools"] and this weather question will trigger the get_weather tool, the execution will pause right before the tools node runs.
At this point, the graph is in a "paused" state waiting for further instruction. 
"""

In [ ]:
snapshot = graph.get_state(config)
existing_message = snapshot.values["messages"][-1]
existing_message.pretty_print()

"""
You're doing exactly these three steps:

snapshot = graph.get_state(config) - Get the current state of the graph for thread ID "4" (which is stored in the config variable)
existing_message = snapshot.values["messages"][-1] - Access the last message in the messages list that's stored in the state. The [-1] is Python's way of accessing the last element in a list.
existing_message.pretty_print() - Display that message in a readable format

At this point, the message you're examining would typically be an AI message that contains a tool call - specifically a call to the weather tool for Munich. This is the message that was generated just before the graph paused execution (because of the interrupt_before=["tools"] setting).
"""

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage


"""
You're creating two new messages and manually inserting them into the conversation state:

ToolMessage: This represents the response from a tool. You're creating a fake response as if the weather tool had returned "It is only 5°C warm today!" instead of its actual response. The important part is that you're connecting it to the original tool call by using 

tool_call_id=existing_message.tool_calls[0]["id"]

which links this response to the specific tool call that was made.

AIMessage: This represents the AI's response after receiving the tool's output. You're setting it to the same text, simulating what the AI might say after getting the tool result.

Then with graph.update_state(), you're injecting these messages into the conversation, effectively overriding what would have happened with the actual tool execution.
"""

answer = "It is only 5°C warm today!"
new_messages = [
    ToolMessage(content=answer, tool_call_id=existing_message.tool_calls[0]["id"]),
    AIMessage(content=answer),
]

""" 
as the last message would the AIMessage with the tool call , in the tool_call_id we are accessing that. 
but the graph was paused before the tool call node execution. 
"""

graph.update_state(
    config,
    {"messages": new_messages},
)

"""
due to the use of Annotated , these new_messages will be appended to the previous messages list.
"""

In [ ]:
print((graph.get_state(config).values["messages"]))

In [ ]:
"""
You're starting a new invocation of the graph (not resuming the paused one)
Since you're providing a new input message, the graph considers this a fresh conversation turn
The graph starts from the beginning (entry point) with this new message
But critically, it's using the same thread ID, so it has access to all previous messages ( inlcluding the fake messages you inserted after pausing the graph execution ) 

The graph doesn't maintain multiple "paused states" within the same thread - when you invoke it with a new message, it treats that as a new turn in the conversation and starts from the beginning.

The original paused state is essentially abandoned when you start a new conversation turn.
"""

"""
so won't the graph pause again at the pause point? i.e before executing 'tool' node.

The LLM now has a complete conversation including what appears to be a previously completed tool call (your injected messages)
When the LLM generates its next response to "How warm was it again?", it likely doesn't need to call a tool again - it can simply respond based on the "temperature information" that appears in the conversation history

Therefore, the graph doesn't hit the pause point because the execution path doesn't lead to tool use. The LLM is responding directly with something like "It was 5°C" based on the conversation history.

If the follow-up question had required a new tool call, then yes, the graph would have paused again before executing that new tool.
and we would have to invoke with None type input with same thread_id to resume the next node execution. 
"""

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="How warm was it again?")

graph.invoke({"messages": input_message}, config=config)

### Replay

In [ ]:
all_checkpoints = []
for state in graph.get_state_history(config=config):
    # get all the history of states of that thread_id.
    all_checkpoints.append(state)
all_checkpoints

In [ ]:
"""
When you select a specific checkpoint with to_replay = all_checkpoints[4], you're selecting both:

The state data as it existed at that historical moment
The execution position in the graph at that historical moment
"""

to_replay = all_checkpoints[4]

to_replay.values #Displays the values in that checkpoint (like messages, state variables, etc.)
#The .values attribute contains all the actual data in that particular state snapshot, similar to how state data is accessed in various nodes across your notebooks.

In [ ]:
to_replay.next #Shows what node would be executed next if you resumed from this checkpoint

In [ ]:
to_replay.config 

In [ ]:
"""
When you use graph.invoke(None, config=to_replay.config), you're resuming execution from the exact node position that's stored in checkpoint 4 - regardless of whether the graph was explicitly paused there or not.
Here's how it works:

If the graph was paused at that position (due to interrupt_before or for any other reason), resuming will continue execution from that paused point.
If the graph was not explicitly paused at that position, but that's just where execution was at the moment the checkpoint was recorded, the system will still resume from that exact node. The checkpoint records the precise state of execution, including which node was about to be executed next.
"""

"""
But if we paused the graph before tool_execution, and haven't resumed it, 
so invoking below from any state, will replay the graph from that historical state and node 
but will 
again come and pause where it was paused before, i.e before the tool execution, as the graph hasn't been resumed yet.
"""

"""
When you access a checkpoint from the exact point where the graph was paused (before tool execution) and invoke with `None`, here's what happens:

If you do `to_replay = all_checkpoints[4]` and that checkpoint represents the state where the graph was paused before tool execution, then when you call `graph.invoke(None, config=to_replay.config)`:

1. The system will recognize that this state was at a pause point (before tools execution)
2. It will resume execution from that pause point
3. It will continue until the next pause point or until completion

It doesn't "double pause" or stay paused - it resumes execution from the pause point and continues running the graph. The `None` input explicitly tells the system "don't add new input, just continue execution from where this state was."

This is different from accessing the current state with `graph.get_state(config)` which would tell you the graph is currently paused but wouldn't resume execution.
"""

graph.invoke(None, config=to_replay.config)

### Branching off past state

In [ ]:
"""
from that accessed index 4 state, we are finding out the last message from the state.
"""

last_message = to_replay.values["messages"][-1]

In [ ]:
last_message.tool_calls

In [ ]:
"""
This modifies the tool call to change which tool will be used
It changes the name from "get_weather" to "broken_api"
This is the key line that creates the branch - we're altering history to see what would happen if a different tool had been called
"""
last_message.tool_calls[0]["name"] = "broken_api"

""" This displays the modified tool calls to confirm our change worked
We can see that the name is now "broken_api" instead of the original value """
last_message.tool_calls

In [ ]:
"""
This updates the state using our modified message
to_replay.config tells it which thread/conversation to update
{"messages": [last_message]} provides the modified message to insert
The return value branch_config is a new configuration that points to this new branched state
This is creating an alternative timeline based on our modification
"""
branch_config = graph.update_state(
    to_replay.config,
    {"messages": [last_message]},
)

In [ ]:
"""
This continues execution using our newly created branch
None means no new input, just resume execution
branch_config points to our modified state that uses "broken_api" instead
This will show the conversation continuing as if the AI had originally called the broken API
"""
graph.invoke(None, branch_config)

In [ ]:
"""
This returns to the original conversation thread by using the original config
It shows that we can explore branches but still return to the main conversation
This demonstrates that branches don't overwrite the original conversation history
"""
graph.invoke(None, config=config)


### Wait for user input

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool

In [ ]:
from langchain_core.tools import tool

"""
This defines a special tool that:

Allows the AI to explicitly request human assistance
Returns an empty string since the actual response will come from the human
The docstring explains to the AI when to use this tool
"""
@tool
def request_assistance():
    """Escalate the conversation to an expert. Use this if you are unable to assist directly or if the user requires support beyond your permissions.

    To use this function, relay the user's 'request' so the expert can provide the right guidance.
    """
    return ""

In [ ]:
tools = [get_weather]
llm = ChatOpenAI(model="gpt-4o-mini")
"""
Note that we're adding request_assistance separately, showing how to combine regular tools with special human-intervention tools
"""
llm_with_tools = llm.bind_tools(tools + [request_assistance])


"""
Gets the LLM's response based on the message history
Sets a flag ask_human to True if the LLM called the request_assistance tool
Returns both the response and the flag
"""
def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    ask_human = False
    if response.tool_calls and response.tool_calls[0]["name"] == "request_assistance":
        ask_human = True


    state["ask_human"] = True
    state['messages'] = response
    return state

    """ this approach below returns a new state not update the existing one """
    # return {"messages": [response], "ask_human": ask_human}

In [ ]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage


"""
This section:

Defines a helper function create_response that formats the human's response as a tool response
Creates a human_node function that:

Checks if there's already a tool response
If not, creates a new ToolMessage with expert advice
Resets the ask_human flag to false
This simulates a human expert providing a specialized response


Adds the human node to the graph
"""
def create_response(response: str, ai_message: AIMessage):
    return ToolMessage(
        content=response,
        tool_call_id=ai_message.tool_calls[0]["id"],
    )


def human_node(state: State):
    new_messages = []
    ''' 
    meaning if aleady no tool has been called and ToolMessage was added to the message list.
    only then append.
    '''
    if not isinstance(state["messages"][-1], ToolMessage):
        new_messages.append(
            create_response(
                "Plan your trip 3 months before and you don't carry a Real Madrid shirt with you",
                state["messages"][-1],
            )
        )

    state["messages"] = new_messages
    state["ask_human"] = False
    return state



graph_builder.add_node("human", human_node)

In [ ]:
"""
This:

Creates a router function that:

Checks if human intervention is needed
If yes, routes to the human node
If no, uses the standard tools_condition to decide


Sets up conditional edges with three possible destinations:

"human" node if human intervention is needed
"tools" node if a regular tool call is made
END if no tools or human intervention needed
"""

def select_next_node(state: State):
    if state["ask_human"]:
        return "human"
    return tools_condition(state)


graph_builder.add_conditional_edges(
    "chatbot",
    select_next_node,
    {"human": "human", "tools": "tools", "__end__": "__end__"},
)

In [ ]:
"""
This:

Adds edges from both tools and human nodes back to chatbot
Sets the entry point to chatbot
Creates a memory saver for conversation history
Compiles the graph with an important addition: interrupt_before=["human"]
The interrupt_before=["human"] is crucial - it pauses execution before the human node runs
This allows a real human to intervene in the workflow
"""

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("human", "chatbot")
graph_builder.set_entry_point("chatbot")
checkpointer = MemorySaver()
graph = graph_builder.compile(
    checkpointer=checkpointer,
    interrupt_before=["human"],
)

In [ ]:
"""
This:

Sets up a configuration with a thread ID
Creates a message asking for travel advice
Invokes the graph with this message
When the AI detects this needs expertise, it will call request_assistance
The graph will pause before the human node
"""

config = {"configurable": {"thread_id": "50"}}
input_message = HumanMessage(
    content="I need some expert advice on how to plan a trip to barcelona"
)
graph.invoke({"messages": input_message}, config=config)

In [ ]:
"""
This:

Resumes execution after the human has reviewed : (in this case it will just answers with a hardcoded human response and convert it to a ToolMessage type, in the human_node )
Continues the conversation flow
Demonstrates how a human can be integrated into the conversation flow
"""
graph.invoke(None, config=config)

### New way Human in the loop - Interrupt + Command

In [ ]:
from typing import TypedDict
from typing_extensions import Literal
from langgraph.graph import END, StateGraph
from langgraph.types import Command


class InputState(TypedDict):
    string_value: str
    numeric_value: int


"""
A helper function that modifies the state by adding "a" to strings and 1 to numbers
Used by the node functions below
"""
def modify_state(input_state: InputState) -> InputState:
    input_state["string_value"] += "a"
    input_state["numeric_value"] += 1
    return input_state


"""
Unlike earlier patterns, this returns a Command object
The Command contains:

goto="branch_b": Where to go next
update=new_state: State changes to apply


This explicitly controls both flow and state in one return value
"""
"""
When you use Command in return types, you're implementing what's called "imperative routing" - you're directly specifying where to go next. 

This implements an edgeless graph. as the Command can tell where to go and update states.
"""

"""
Here's what the Literal["branch_b"] means and why it's significant:

Type Safety and Validation: It declares that this function can only return a Command that routes to "branch_b" and no other destination. This acts as a compile-time check to ensure the node only navigates to allowed destinations.

Avoiding Runtime Errors: It helps prevent errors where a node might try to navigate to a destination that doesn't exist or isn't connected in the graph.

The Literal type ensures that the dynamic routing done with Command still follows the statically defined structure of your graph, combining the flexibility of runtime decisions with the safety of compile-time checks.
"""
def branch_a(state: InputState) -> Command[Literal["branch_b"]]:
    print(f"branch_a: Current state: {state}")

    """
    Since the Command API is designed to work with new states, emitting a new state (update=new_state) is the correct approach here.
    Updating the existing state directly would bypass the Command mechanism and could lead to unexpected behavior.
    """
    new_state = modify_state(state)
    print(f"branch_a: Updated state: {new_state}")
    return Command(
        goto="branch_b",
        update=new_state,
    )


def branch_b(state: InputState) -> Command[Literal["branch_c"]]:
    print(f"branch_b: Current state: {state}")
    new_state = modify_state(state)
    print(f"branch_b: Updated state: {new_state}")
    return Command(
        goto="branch_c",
        update=new_state,
    )


def branch_c(state: InputState) -> Command[Literal[END]]:
    print(f"branch_c: Current state: {state}")
    new_state = modify_state(state)
    print(f"branch_c: Updated state: {new_state}")
    return Command(goto=END, update=new_state)

In [ ]:
graph = StateGraph(InputState)

graph.add_node("branch_a", branch_a)
graph.add_node("branch_b", branch_b)
graph.add_node("branch_c", branch_c)
graph.set_entry_point("branch_a")

runnable = graph.compile()

In [ ]:
initial_state = {"string_value": "Hello", "numeric_value": 0}
final_state = runnable.invoke(initial_state)

print("Final state:", final_state)

In [ ]:
from typing_extensions import Literal
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from IPython.display import Image, display


@tool
def weather_search(city: str):
    """Search for the weather in a specific city"""
    print("----")
    print(f"Searching for: {city}")
    print("----")
    return "Sunny!"


model = ChatOpenAI(model="gpt-4o-mini").bind_tools([weather_search])


def call_llm(state):
    return {"messages": [model.invoke(state["messages"])]}



def human_review_node(state) -> Command[Literal["call_llm", "run_tool"]]:
    """
    Gets the last message and its tool call
    Key new feature: Uses the interrupt() function to pause execution
    Provides context for the human reviewer (the question and tool call)
    This actively pauses execution and waits for human input
    """
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[-1]

    human_review = interrupt(
        {
            "question": "Is this correct?",
            "tool_call": tool_call,
        }
    )
    print("Human review values:", human_review)

    """
    Gets the human's decision (continue or update)
    If "continue", uses Command to direct flow to run_tool without changes
    """
    review_action = human_review["action"]
    review_data = human_review.get("data")

    if review_action == "continue":
        return Command(goto="run_tool")


    elif review_action == "update":
        """
        If "update", creates a modified message with the human's corrections
        Returns a Command that both:

        Routes to the run_tool node
        Updates the state with the corrected message


        This allows humans to modify parameters before tool execution
        """
        updated_message = {
            "role": "ai",
            "content": last_message.content,
            "tool_calls": [
                {
                    "id": tool_call["id"],
                    "name": tool_call["name"],
                    "args": review_data,
                }
            ],
            "id": last_message.id,
        }
        return Command(goto="run_tool", update={"messages": [updated_message]})


def route_after_llm(state) -> Literal[END, "human_review_node"]:
    if len(state["messages"][-1].tool_calls) == 0:
        return END
    else:
        return "human_review_node"


builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)
builder.add_node("run_tool", ToolNode(tools=[weather_search]))
builder.add_node("human_review_node", human_review_node)

"""
only adding edges where return type is not Command, as Command type can decide where to go itself, we don't need edges. 
as you can see in the human_review_node.
"""
builder.add_conditional_edges("call_llm", route_after_llm)
builder.add_edge("run_tool", "call_llm")
builder.set_entry_point("call_llm")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
"""
Starts a conversation about weather
Note the typo in "Munic" (not "Munich")
"""
initial_input = {"messages": [HumanMessage(content="How is the weather in Munic?")]}
config = {"configurable": {"thread_id": "5"}}

first_result = graph.invoke(initial_input, config=config, stream_mode="updates")
first_result

In [ ]:

#Shows that execution is waiting at the human review node
print(graph.get_state(config).next)

In [ ]:
"""
Resumes execution, choosing to continue without changes
The weather search will run with "Munic" (incorrect)
"""
graph.invoke(Command(resume={"action": "continue"}), config=config)

In [ ]:

"""
Starts a new conversation with the same typo with a new thread_id.
But this time we'll correct it
"""
config = {"configurable": {"thread_id": "6"}}

graph.invoke(initial_input, config=config, stream_mode="updates")


In [ ]:
print(graph.get_state(config).next)

In [ ]:

"""
Resumes execution but with corrected data
This shows how humans can fix errors before tool execution
The Command with resume provides structured data for the correction
"""
graph.invoke(Command(resume={"action": "update", "data": {"city": "Munich"}}), config)